# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from sec import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "YUM"
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Yum Brands Inc.',
 'marketcap': 35777175861,
 'week52high': 132.24,
 'week52low': 103.49,
 'week52highSplitAdjustOnly': 132.87,
 'week52lowSplitAdjustOnly': 103.97,
 'week52change': 0.03337876044993182,
 'sharesOutstanding': 281687866,
 'float': 0,
 'avg10Volume': 1611668,
 'avg30Volume': 1419182,
 'day200MovingAvg': 119.92,
 'day50MovingAvg': 128.73,
 'employees': 2020,
 'ttmEPS': 4.4,
 'ttmDividendRate': 2.263877104969058,
 'dividendYield': 0.017824400480033524,
 'nextDividendDate': '',
 'exDividendDate': '2022-11-23',
 'nextEarningsDate': '2023-02-08',
 'peRatio': 27.86384412823988,
 'beta': 0.6896769291288999,
 'maxChangePercent': 6.872195363827941,
 'year5ChangePercent': 0.7319506092031609,
 'year2ChangePercent': 0.24948966792753532,
 'year1ChangePercent': 0.03337876044993182,
 'ytdChangePercent': -0.0083541536539663,
 'month6ChangePercent': 0.08273673837172635,
 'month3ChangePercent': 0.04772636531919328,
 'month1ChangePercent': -0.005403288958496511,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data["year1ChangePercent"]

0.03337876044993182

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        try: 
             append_dataframe = pd.DataFrame([[symbol, 
                                               data[symbol]['quote']['latestPrice'], 
                                               data[symbol]['stats']['year1ChangePercent'],
                                               'N/A']], 
                                      columns= my_columns)
             final_dataframe = pd.concat([final_dataframe,append_dataframe], ignore_index=True)
        except KeyError:
            print(f"stock{symbol} is not on the list")   
    

stockDISCA is not on the list
stockHFC is not on the list
stockVIAC is not on the list
stockWLTW is not on the list


In [7]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,152.570,0.102373,N/A
1,AAL,16.940,0.03465,N/A
2,AAP,150.560,-0.305482,N/A
3,AAPL,152.000,-0.098586,N/A
4,ABBV,145.990,0.071647,N/A
...,...,...,...,...
496,YUM,128.610,0.033379,N/A
497,ZBH,128.570,0.062685,N/A
498,ZBRA,324.835,-0.342444,N/A
499,ZION,53.770,-0.21444,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop=True, inplace= True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.920,2.281287,N/A
1,FTI,13.115,0.935199,N/A
2,HRB,39.355,0.694134,N/A
3,NUE,170.350,0.613898,N/A
4,UNM,42.370,0.585593,N/A
5,LW,99.190,0.583158,N/A
6,VAR,177.070,0.571162,N/A
7,CAH,77.115,0.562606,N/A
8,VLO,128.055,0.556025,N/A
9,MPC,116.150,0.551055,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
portfolio_input()
print(portfolio_size)

1000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0,len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, "Number of Shares to Buy"] =  math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.920,2.281287,245
1,FTI,13.115,0.935199,1495
2,HRB,39.355,0.694134,498
3,NUE,170.350,0.613898,115
4,UNM,42.370,0.585593,462
5,LW,99.190,0.583158,197
6,VAR,177.070,0.571162,110
7,CAH,77.115,0.562606,254
8,VLO,128.055,0.556025,153
9,MPC,116.150,0.551055,168


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
              'Ticker',
              'Price',
              'Number of Shares to Buy',
              'One-Year Price Return',
              'One-Year Return Percentile',
              'Six-Month Price Return',
              'Six-Month Return Percentile',
              'Three-Month Price Return',
              'Three-Month Return Percentile',
              'One-Month Price Return',
              'One-Month Return Percentile',
              'HQM Score'
              ]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
convert_none = lambda x : 0 if x is None else x

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        try:
            append_dataframe = pd.DataFrame([[symbol, 
                                              convert_none(data[symbol]['quote']['latestPrice']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['year1ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month6ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month3ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month1ChangePercent']),
                                              'N/A',
                                              'N/A']], columns=hqm_columns)
            hqm_dataframe= pd.concat([hqm_dataframe,append_dataframe], ignore_index=True)
        except KeyError:
            print(f'Stock {symbol} isnt in the list anymore')

Stock DISCA isnt in the list anymore
Stock HFC isnt in the list anymore
Stock VIAC isnt in the list anymore
Stock WLTW isnt in the list anymore


In [12]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.600,N/A,0.102373,N/A,0.160455,N/A,0.137422,N/A,0.016041,N/A,N/A
1,AAL,16.940,N/A,0.034650,N/A,0.154681,N/A,0.206237,N/A,0.216583,N/A,N/A
2,AAP,150.530,N/A,-0.305482,N/A,-0.197878,N/A,-0.150727,N/A,-0.006464,N/A,N/A
3,AAPL,151.960,N/A,-0.098586,N/A,-0.062887,N/A,0.117901,N/A,0.235802,N/A,N/A
4,ABBV,146.000,N/A,0.071647,N/A,0.072673,N/A,0.008950,N/A,-0.103430,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.605,N/A,0.033379,N/A,0.082737,N/A,0.047726,N/A,-0.005403,N/A,N/A
497,ZBH,128.550,N/A,0.062685,N/A,0.147675,N/A,0.236963,N/A,0.026729,N/A,N/A
498,ZBRA,324.835,N/A,-0.342444,N/A,-0.002683,N/A,0.434811,N/A,0.272053,N/A,N/A
499,ZION,53.770,N/A,-0.214440,N/A,0.022915,N/A,0.102361,N/A,0.102533,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.708583
1      0.590818
2      0.053892
3      0.303393
4      0.658683
         ...   
496    0.588822
497    0.646707
498    0.035928
499    0.129741
500    0.225549
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.768463
1      0.752495
2      0.033932
3      0.217565
4      0.570858
         ...   
496    0.596806
497    0.732535
498    0.323353
499     0.43513
500    0.279441
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.610778
1      0.766467
2      0.015968
3      0.550898
4      0.241517
         ...   
496    0.357285
497    0.814371
498    0.974052
499    0.508982
500    0.848303
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.325349
1      0.918164
2      0.211577
3      0.944112
4      0.013972
         ...   
496    0.215569
497    0.359281
498    0.964072
499    0.644711
500    0.818363
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.600,N/A,0.102373,0.708583,0.160455,0.768463,0.137422,0.610778,0.016041,0.325349,N/A
1,AAL,16.940,N/A,0.034650,0.590818,0.154681,0.752495,0.206237,0.766467,0.216583,0.918164,N/A
2,AAP,150.530,N/A,-0.305482,0.053892,-0.197878,0.033932,-0.150727,0.015968,-0.006464,0.211577,N/A
3,AAPL,151.960,N/A,-0.098586,0.303393,-0.062887,0.217565,0.117901,0.550898,0.235802,0.944112,N/A
4,ABBV,146.000,N/A,0.071647,0.658683,0.072673,0.570858,0.008950,0.241517,-0.103430,0.013972,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.605,N/A,0.033379,0.588822,0.082737,0.596806,0.047726,0.357285,-0.005403,0.215569,N/A
497,ZBH,128.550,N/A,0.062685,0.646707,0.147675,0.732535,0.236963,0.814371,0.026729,0.359281,N/A
498,ZBRA,324.835,N/A,-0.342444,0.035928,-0.002683,0.323353,0.434811,0.974052,0.272053,0.964072,N/A
499,ZION,53.770,N/A,-0.214440,0.129741,0.022915,0.43513,0.102361,0.508982,0.102533,0.644711,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [15]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [16]:
hqm_dataframe.sort_values(by='HQM Score', ascending=False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [17]:
portfolio_input()

In [19]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.600,128,0.102373,0.708583,0.160455,0.768463,0.137422,0.610778,0.016041,0.325349,0.603293
1,AAL,16.940,1157,0.034650,0.590818,0.154681,0.752495,0.206237,0.766467,0.216583,0.918164,0.756986
2,AAP,150.530,130,-0.305482,0.053892,-0.197878,0.033932,-0.150727,0.015968,-0.006464,0.211577,0.078842
3,AAPL,151.960,129,-0.098586,0.303393,-0.062887,0.217565,0.117901,0.550898,0.235802,0.944112,0.503992
4,ABBV,146.000,134,0.071647,0.658683,0.072673,0.570858,0.008950,0.241517,-0.103430,0.013972,0.371257
5,ABC,155.230,126,0.165326,0.816367,0.119499,0.684631,-0.024341,0.145709,-0.038938,0.097804,0.436128
6,ABMD,381.020,51,0.000000,0.492016,0.000000,0.354291,0.000000,0.198603,0.000000,0.248503,0.323353
7,ABT,111.060,176,-0.122819,0.269461,0.032733,0.459082,0.145061,0.632735,0.013503,0.313373,0.418663
8,ACN,284.840,68,-0.155098,0.223553,-0.057073,0.235529,0.111974,0.540918,0.104278,0.648703,0.412176
9,ADBE,376.170,52,-0.261343,0.077844,-0.124818,0.081836,0.327489,0.926148,0.154945,0.816367,0.475549


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [20]:
writer= pd.ExcelWriter('momentum_percentile.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [22]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [23]:
writer.save()

C:\Users\ttt\AppData\Local\Temp\ipykernel_18968\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
